### OPERATIONS HAUT NIVEAU

In [ ]:
import pandas as pd
import numpy as np
from time import time
rng = np.random.default_rng(seed=int(time()))
pd.__version__

In [ ]:
url = "http://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv"
penguins_df = pd.read_csv(
    url,
    encoding="utf8"
)
penguins_df

#### GROUP BY

In [ ]:
# poids moyens des penguins par espèce ou sexe
# => slectionnner les champs utiles
# => les champs du groupby doivent être dans le df (comme en SQL)
gb = penguins_df[["body_mass_g","species","sex"]].groupby(by=["species","sex"])
# gb["body_mass_g"].mean()
grouped_df = gb["body_mass_g"].agg(["mean", "max"])
grouped_df

In [ ]:
# Notion d'index multiple: requêtage
grouped_df.loc["Adelie", "mean"]
grouped_df.loc[("Adelie", "FEMALE")]

In [ ]:
# manipulation
grouped_df.swaplevel(1,0, axis=0)

#### SEGMENTATION

In [ ]:
# on veut repartir les penguins par catégories de poids i.e avec des intervalles min et max
# on veut 3 catégories arbitraires légers, medium et lourds avec des seuils linéaires i.e longueur d'intervalle égales
# => max - min égales => |----------|----------|----------|
#                       min  léger    médium       lourd   max
penguins_df 

In [ ]:
# 5 intervalles de même longueur entre 0 et 100
np.linspace(0, 100, num=5)
lin = np.linspace(np.sqrt(np.pi**2), np.sqrt(6*np.pi**2), num=5)
np.diff(lin)

In [ ]:
# on peut unpacker les series
w_min, w_max = penguins_df["body_mass_g"].agg(["min", "max"])
# 3 catégories donc 4 seuils
intervals = np.linspace(w_min, w_max, num=4)
print(intervals)
body_mass_category = pd.cut(
    penguins_df["body_mass_g"], 
    bins=intervals,
    labels=["light", "medium", "heavy"],
    # gère si le min (resp. max) est exclu de l'intervalle
    right=False
)
penguins_df.insert(
    6,
    value=body_mass_category, 
    column="body_mass_category"
)

In [ ]:
penguins_df.groupby(["body_mass_category", "sex"])["species"].count()

In [ ]:
# segmentation avec des quantiles
penguins_df["body_mass_quantile"] = pd.qcut(
    penguins_df["body_mass_g"],
    q=[0, 0.33, 0.66, 1],
    labels=["q_light", "q_medium", "q_heavy"]
)


In [ ]:
penguins_df.groupby(["body_mass_quantile", "sex"])["species"].count()

#### PIVOT AKA TABLEAU CROISE DYNAMIQUE

##### pivot
* fabrication du df:
  1. une colonne student avec 10 valeurs student_{i} répétées 4 fois 
  2. une colonne subject avec 4 valeurs ["maths", "english", "biology", "physics"]["maths", "english", "biology", "physics"]  répétées dans l'ordre 10 fois
  3. une colonne note avec des entiers entre 0 et 20 (avec les 0.5 possibles)
  4. une colonne coeff qui répètent les mêmes valeurs en fonction du subject



In [121]:
students = [ f"student_{i}" for i in range(1, 11)]
subjects = ["maths", "english", "biology", "physics"]
coeffs = [4,3,2,3]
multi_students = np.repeat(students, repeats=4)
multi_subjects = np.tile(subjects, reps=10) # aka subjects * 10
multi_coeffs = np.tile(coeffs, reps=10)
notes = rng.integers(0, 40, endpoint=True, size=40) / 2
notes_df = pd.DataFrame(
    data={
        # technique par répétition
        # "student": multi_students,
        # "subject": multi_subjects,
        "coeff": multi_coeffs,
        "note": notes
    },
    # OU technique par un index multiple généré avec le produit cartésiens des listes
    index=pd.MultiIndex.from_product([students, subjects])
)
# format long i.e: les colonnes ont des valeurs qui se répètent bcp et on a relativment moins de colonnes 
notes_df = notes_df.reset_index().rename(columns={
    "level_0": "student",
    "level_1": "subject",
})
notes_df

,student,subject,coeff,note
0,student_1,maths,4,8.5
1,student_1,english,3,9.5
2,student_1,biology,2,17.5
3,student_1,physics,3,8.5
4,student_2,maths,4,16.0
5,student_2,english,3,7.0
6,student_2,biology,2,8.0
7,student_2,physics,3,13.0
8,student_3,maths,4,19.0
9,student_3,english,3,14.5


In [123]:
# pour changer en format large
# on va dispatcher la colonne en tant que 4 colones "maths", "english", "biology", "physics"
# dont les valeurs vont venir depuis la colonne note
pivoted_df = notes_df.pivot(
    # la ou les colonnes à dégager et à dipatcher dans les nouvelles colonnes dont le nom sont les valeurs
    columns="subject",
    # la ou les colonnes qui vont alimenter les nouvelles colonnes
    values=["note", "coeff"],
    index="student"
)
pivoted_df

note                         coeff                      
subject    biology english maths physics biology english maths physics
student                                                               
student_1     17.5     9.5   8.5     8.5     2.0     3.0   4.0     3.0
student_10     7.0     0.5  14.5    15.0     2.0     3.0   4.0     3.0
student_2      8.0     7.0  16.0    13.0     2.0     3.0   4.0     3.0
student_3     10.5    14.5  19.0     1.0     2.0     3.0   4.0     3.0
student_4      3.0     0.0   9.5    10.0     2.0     3.0   4.0     3.0
student_5      1.5     4.5  13.0    12.0     2.0     3.0   4.0     3.0
student_6      0.0     6.5   1.5     3.0     2.0     3.0   4.0     3.0
student_7      7.5    19.5   9.5    14.5     2.0     3.0   4.0     3.0
student_8      3.0     9.0   0.0    10.0     2.0     3.0   4.0     3.0
student_9      1.5    17.5  15.5     1.0     2.0     3.0   4.0     3.0

In [124]:
## plusieurs de colonnes
pivoted_df.loc["student_1", "note"]
pivoted_df.loc["student_1", ("note", "biology")]
print(pivoted_df.loc["student_1", ("note", ["biology", "maths"])])
# moyenne pondérée de chaque étudiant
np.average(a=pivoted_df["note"], weights=pivoted_df["coeff"], axis=1)
pivoted_df.apply(lambda row: np.average(row["note"], weights=row["coeff"]), axis=1)
# pivoted_df.loc["student_1", ("note", "biology")]

      subject
note  biology    17.5
      maths       8.5
Name: student_1, dtype: float64


student
student_1     10.250000
student_10     9.875000
student_2     11.666667
student_3     11.958333
student_4      6.166667
student_5      8.708333
student_6      2.875000
student_7     12.916667
student_8      5.250000
student_9     10.041667
dtype: float64

In [125]:
## revenir du format large vers le format long
# stack() si on a plusieurs de colonnes on peut pivoter niveau en tant que nouveau index
# reset_index() : remettre un ou plusieurs de niveaux d'index en tant que colonne
notes_df = pivoted_df.stack(level=1).reset_index()
notes_df

,student,subject,note,coeff
0,student_1,biology,17.5,2.0
1,student_1,english,9.5,3.0
2,student_1,maths,8.5,4.0
3,student_1,physics,8.5,3.0
4,student_10,biology,7.0,2.0
5,student_10,english,0.5,3.0
6,student_10,maths,14.5,4.0
7,student_10,physics,15.0,3.0
8,student_2,biology,8.0,2.0
9,student_2,english,7.0,3.0


In [105]:
## inverser un pivot (large => long) avec un seul index de colonne
# multiple comprehension lists
new_cols = [ subject + "_" + item for subject in pivoted_df.columns.levels[1] for item in pivoted_df.columns.levels[0]]
# tri custom sur une partie de la chaine
new_cols.sort(
    key=lambda v: v[v.index("_") + 1:], 
    reverse=True)
# ici le rename ne marche pas
# pivoted_df.rename(columns=???)
pivoted_df = pivoted_df.droplevel(axis=1, level=0)
# écraser l'index
pivoted_df.columns = pd.Index(new_cols)
pivoted_df.reset_index(inplace=True)
pivoted_df

,student,biology_note,english_note,maths_note,physics_note,biology_coeff,english_coeff,maths_coeff,physics_coeff
0,student_1,12.0,8.0,17.0,12.5,2.0,3.0,4.0,3.0
1,student_10,4.0,5.5,0.5,2.0,2.0,3.0,4.0,3.0
2,student_2,5.5,13.0,1.0,2.5,2.0,3.0,4.0,3.0
3,student_3,4.0,5.5,13.5,10.0,2.0,3.0,4.0,3.0
4,student_4,14.0,7.5,3.5,10.0,2.0,3.0,4.0,3.0
5,student_5,16.0,18.5,15.5,17.0,2.0,3.0,4.0,3.0
6,student_6,5.5,7.5,20.0,20.0,2.0,3.0,4.0,3.0
7,student_7,4.0,19.5,3.5,19.0,2.0,3.0,4.0,3.0
8,student_8,5.0,3.0,10.5,20.0,2.0,3.0,4.0,3.0
9,student_9,6.5,15.5,15.5,14.0,2.0,3.0,4.0,3.0


In [120]:
# fusionner les notes et ensuite fusionner les coeffs

note_df = pivoted_df.loc[:, "student":"physics_note"]
note_df.columns = pd.Index(["student"] + subjects)
note_df = note_df.melt(
    # colonnes indépendantes à la fusion
    id_vars="student",
    # conserver l'index
    # ignore_index=True
    # le nom de la nouvelle colonne qui va aggréger les colonnes anciennes
    var_name="subject",
    # la colonne des anciennes valeurs des anciennes colonnes
    value_name="note"
)
coeff_df = pivoted_df.loc[:, "biology_coeff":]
coeff_df.columns = pd.Index(subjects)
coeff_df = coeff_df.melt(
    var_name="subject",
    value_name="coeff"
)
coeff_df
# pd.concat([note_df, coeff_df], axis=1)
# jointure SQL
# pd.merge(
#     left=note_df,
#     right=coeff_df,
#     how="inner",
#     on="subject")
note_df.merge(
    right=coeff_df,
    how="inner",
    on="subject"
)
    


,student,subject,note,coeff
0,student_1,maths,12.0,2.0
1,student_1,maths,12.0,2.0
2,student_1,maths,12.0,2.0
3,student_1,maths,12.0,2.0
4,student_1,maths,12.0,2.0
...,...,...,...,...
395,student_9,physics,14.0,3.0
396,student_9,physics,14.0,3.0
397,student_9,physics,14.0,3.0
398,student_9,physics,14.0,3.0
